In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import lightgbm as lgb
import scipy as sp
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from tqdm import tqdm

In [2]:
tqdm.pandas()

/home/jordan/anaconda3/envs/tf2/lib/python3.7/site-packages/tqdm/std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [3]:
%%time
# Only load those columns in order to save space
keep_cols = ['event_id', 'game_session', 'installation_id', 'event_count', 'event_code', 'title', 'game_time', 'type', 'world']

train = pd.read_csv('./source_data/train.csv', usecols=keep_cols)
test = pd.read_csv('./source_data/test.csv', usecols=keep_cols)
train_labels = pd.read_csv('./source_data/train_labels.csv')
submission = pd.read_csv('./source_data/sample_submission.csv')

CPU times: user 15.9 s, sys: 988 ms, total: 16.9 s
Wall time: 15.3 s


In [5]:
test_assess = test[test.type == 'Assessment'].copy()
test_labels = submission.copy()
test_labels['title'] = test_labels.installation_id.progress_apply(
    lambda install_id: test_assess[test_assess.installation_id == install_id].iloc[-1].title
)

  0%|          | 0/1000 [00:00<?, ?it/s]

  2%|▏         | 17/1000 [00:00<00:06, 162.82it/s]

  4%|▍         | 41/1000 [00:00<00:05, 179.31it/s]

  6%|▋         | 65/1000 [00:00<00:04, 192.91it/s]

  9%|▉         | 89/1000 [00:00<00:04, 203.53it/s]

...

 92%|█████████▏| 921/1000 [00:03<00:00, 243.86it/s]

 95%|█████████▍| 946/1000 [00:03<00:00, 244.14it/s]

 97%|█████████▋| 971/1000 [00:03<00:00, 244.25it/s]

100%|█████████▉| 996/1000 [00:04<00:00, 244.66it/s]

100%|██████████| 1000/1000 [00:04<00:00, 242.88it/s]

In [6]:
def compute_group1_stats(group1, col):
    return group1[
        ['installation_id', col, 'event_count', 'game_time']
    ].groupby(['installation_id', col]).agg(
        [np.mean, np.sum, np.std]
    ).reset_index().pivot(
        columns=col,
        index='installation_id'
    )


In [7]:
def group_and_reduce(df, df_labels):
    """
    Author: https://www.kaggle.com/xhlulu/
    Source: https://www.kaggle.com/xhlulu/ds-bowl-2019-simple-lgbm-using-aggregated-data
    """

    # First only filter the useful part of the df
    df = df[df.installation_id.isin(df_labels.installation_id.unique())]

    # group1 and group2 are intermediary "game session" groups,
    # which are reduced to one record by game session. group1 takes
    # the max value of game_time (final game time in a session) and 
    # of event_count (total number of events happened in the session).
    # group2 takes the total number of event_code of each type
    group1 = df.drop(columns=['event_id', 'event_code']).groupby(
        ['game_session', 'installation_id', 'title', 'type', 'world']
    ).max().reset_index()

    group2 = pd.get_dummies(
        df[['game_session', 'installation_id', 'event_code']],
        columns=['event_code']
    ).groupby(['game_session', 'installation_id']).sum().reset_index()

    # group3, group4, group5 are grouped by installation_id 
    # and reduced using summation and other summary stats
    group3 = pd.get_dummies(
        group1.drop(columns=['game_session', 'event_count', 'game_time']),
        columns=['title', 'type', 'world']
    ).groupby(['installation_id']).sum()

    group4 = group1[
        ['installation_id', 'event_count', 'game_time']
    ].groupby(
        ['installation_id']
    ).agg([np.sum, np.mean, np.std, np.min, np.max])

    # Additional stats on group1
    world_time_stats = compute_group1_stats(group1, 'world')
    type_time_stats = compute_group1_stats(group1, 'type')

    return group3.join(group4).join(
        world_time_stats).join(type_time_stats).fillna(0)


In [8]:
%%time
train_small = group_and_reduce(train, train_labels)
test_small = group_and_reduce(test, test_labels)

print(train_small.shape)
train_small.head()

/home/jordan/anaconda3/envs/tf2/lib/python3.7/site-packages/pandas/core/reshape/merge.py:617: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
/home/jordan/anaconda3/envs/tf2/lib/python3.7/site-packages/pandas/core/reshape/merge.py:617: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 3 on the right)
  warnings.warn(msg, UserWarning)


(3614, 110)
CPU times: user 15.2 s, sys: 2.71 s, total: 17.9 s
Wall time: 8.77 s


,title_12 Monkeys,title_Air Show,title_All Star Sorting,title_Balancing Act,title_Bird Measurer (Assessment),title_Bottle Filler (Activity),title_Bubble Bath,title_Bug Measurer (Activity),title_Cart Balancer (Assessment),title_Cauldron Filler (Assessment),...,"(game_time, mean, Clip)","(game_time, mean, Game)","(game_time, sum, Activity)","(game_time, sum, Assessment)","(game_time, sum, Clip)","(game_time, sum, Game)","(game_time, std, Activity)","(game_time, std, Assessment)","(game_time, std, Clip)","(game_time, std, Game)"
installation_id,,,,,,,,,,,,,,,,,,,,,
0006a69f,2.0,2.0,4.0,0.0,2.0,2.0,2.0,2.0,0.0,0.0,...,0.0,106966.450000,3199695.0,236429.0,0.0,2139329.0,350054.566401,28330.303185,0.0,58189.254197
0006c192,1.0,0.0,0.0,2.0,1.0,1.0,1.0,1.0,0.0,2.0,...,0.0,88345.500000,1210530.0,323061.0,0.0,530073.0,127422.782500,98940.202632,0.0,62500.291205
00129856,0.0,0.0,0.0,1.0,1.0,2.0,0.0,2.0,0.0,0.0,...,0.0,0.000000,1021179.0,39742.0,0.0,0.0,130499.803239,28043.854942,0.0,0.000000
001d0ed0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,2.0,0.0,...,0.0,158426.166667,92282.0,201941.0,0.0,950557.0,24694.997226,17737.374861,0.0,123969.846618
00225f67,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,129984.750000,294517.0,35637.0,0.0,519939.0,49028.831364,12301.536672,0.0,65432.543128


In [13]:
print(train_small.shape)
print(train_small.head())

(3614, 110)
                 title_12 Monkeys  title_Air Show  title_All Star Sorting  \
installation_id                                                             
0006a69f                      2.0             2.0                     4.0   
0006c192                      1.0             0.0                     0.0   
00129856                      0.0             0.0                     0.0   
001d0ed0                      0.0             0.0                     0.0   
00225f67                      1.0             1.0                     1.0   

                 title_Balancing Act  title_Bird Measurer (Assessment)  \
installation_id                                                          
0006a69f                         0.0                               2.0   
0006c192                         2.0                               1.0   
00129856                         1.0                               1.0   
001d0ed0                         2.0                               1.0   
0022

In [14]:
titles = train_labels.title.unique()
title2mode = {}

for title in titles:
    mode = train_labels[train_labels.title == title].accuracy_group.value_counts().index[0]
    title2mode[title] = mode

train_labels['title_mode'] = train_labels.title.apply(lambda title: title2mode[title])
test_labels['title_mode'] = test_labels.title.apply(lambda title: title2mode[title])